In [1]:
"""
(Re-)Created on October 26 2018
@author: Neven Caplar
@contact: ncaplar@princeton.edu
"""

'\nCreated on October 26 2018\n@author: Neven Caplar\n@contact: ncaplar@princeton.edu\n'

In [2]:
# Enables the Cython "magic" command %%cython
%load_ext Cython

In [ ]:
# In cell below you need to specify where the lgsl and lgslcblas libraries can be found
# For me this is within lsst_stack installation
# -I means "include"
# -L means "libraries"




In [12]:
%%cython -lgsl -lgslcblas -I /Users/nevencaplar/Documents/lsst_stack/stack/miniconda3-4.3.21-10a4fa6/DarwinX86/gsl/2.4/include/ -L /Users/nevencaplar/Documents/lsst_stack/stack/miniconda3-4.3.21-10a4fa6/DarwinX86/gsl/2.4/lib/ --cplus --annotate

try:
    from functools import lru_cache
except ImportError:
    from backports.functools_lru_cache import lru_cache 
    
import collections
import functools
import pickle

memoized = []


def clearAllMemoized():
    [m.reset() for m in memoized]


class memoize(object):

    '''Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    '''

    def __init__(self, func):
        self.func = func
        self.cache = {}
        memoized.append(self)

    def __call__(self, *args):
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value

    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__

    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

    def reset(self):
        self.cache = {}
    def show_cache(self):
        print(self.cache)
    def save_obj(self, name):
        with open('/Users/nevencaplar/Documents/PFS/Cython/'+ name + '.pkl', 'wb') as f:
            pickle.dump(self.cache, f, pickle.HIGHEST_PROTOCOL)
    def load_obj(self,name):
        with open('/Users/nevencaplar/Documents/PFS/Cython/' + name + '.pkl', 'rb') as f:
            self.cache=pickle.load(f)
    
    
@memoize
def T_Cython_PythonLayer(n,m,radius,f):
    return T_Cython(n,m,radius,f)


import numpy as np # access to Numpy from Python layer
cimport numpy as np # access to Numpy from Cython layer

#from scipy.special import binom
#from scipy.special import comb
#from scipy.special import jn
from libc.math cimport abs
from libc.math cimport tgamma
from libc.math cimport pow
from libc.math cimport sqrt
from libc.math cimport round
from libc.math cimport trunc

#import gsl_Neven
cimport gsl_Neven
#gsl_sf_poch.gsl_sf_bessel_jl_cython(2,2)
#from gsl_sf_poch import gsl_sf_bessel_jl_cython
#from gsl_sf_poch import gsl_sf_bessel_Jn_cython
#from gsl_sf_poch import gsl_sf_poch_cython


cimport cython


#@cython.boundscheck(False) # won't check that index is in bounds of array
#@cython.wraparound(False) # array[-1] won't work
#@cython.nonecheck(False) # variables are never set to None
#@cython.cdivision(True) # don't protect against dividing by zero

#cpdef double binomialCoeff_Cython_old(double n , double k):
#    return tgamma(n+1)/(tgamma(k+1)*tgamma(n-k+1))

#@cython.nonecheck(False)
#@cython.cdivision(True)
#cdef double gsl_sf_bessel_Jn_cython(int n, double x): 
#    return gsl.gsl_sf_bessel_Jn_cython(n, x)

#@cython.nonecheck(False)
#@cython.cdivision(True)
#cdef double gsl_sf_bessel_jl_cython(int n, double x): 
#    return gsl.gsl_sf_bessel_jl_cython(n, x)

#@cython.nonecheck(False)
#@cython.cdivision(True)
#cdef double gsl_sf_poch_cython(double n , double k): 
#    return gsl.gsl_sf_poch_cython(n,k)

@cython.nonecheck(False)
@cython.cdivision(True)
cdef double binomialCoeff_Cython(int n , int k):
    return gsl_Neven.gsl_sf_poch(1-k+n,k)/tgamma(k+1)

#cpdef j_fun_Cython(int k,double z):
#    cdef:
#        double nominator=3.14159265358979323846
#        double denominator=2*z
#        double kprime
#    kprime=k+0.5
#    return sqrt(nominator/denominator)*jn(kprime,z)

cdef extern from "complex.h":
    double complex exp(double complex)

cdef double is_odd(int num):
    return num & 0x1

cdef double epsilon(int m):
    if m<0 and is_odd(m):
        return -1
    else:
        return 1 

cpdef double complex sub_sum_Cython(double complex prefactor,int nr,int m,double radius,double theta,double f):
    '''sub_sum_Cython, input is double complex prefactor,int nr,int m,double radius,double theta,double f'''
    if prefactor==0:
        res=0+0j
    else:
        res=prefactor*(1j)**m*T_Cython_PythonLayer(nr,abs(m),radius,f)*exp(1j*m*theta)
    return res   

cdef double complex T_Cython(int n, int m, double radius, double f):
    cdef:
        int p
        # p_end is the variable which is called k in Equation 15
        int p_end = int((n-m)/2)
        double complex sum_value=0+0j
    for p in range(p_end+1):
        #change in the order of variables n and m here!
        sum_value+=h_Cython(m,p_end,p)*V_large_defocus_Cython_fake(m,m+2*p, radius, f)
    return sum_value

@cython.cdivision(True)    
cdef double h_Cython(int m, int k, int p):
    cdef:
        double nominator=m+2*p+1
        double denominator=m+p+k+1
    return (nominator/denominator)*binomialCoeff_Cython(k,p)/binomialCoeff_Cython(m+k+p,p)

@cython.cdivision(True)    
cdef double complex V_large_defocus_Cython_fake(int m,int n,double r,double f):
    cdef:
        int p=(n-abs(m))/2
        int q=(n+abs(m))/2
        int kmax=(abs(<int>(f*1))+abs(<int>(r*0.7))+5)
        double complex res=0+0j

    res=epsilon(m)*exp(1j*f/2)*OuterSum_large_defocus_Cython_fake(m,n,r,f,kmax)
    return res 

@cython.cdivision(True)
cpdef double complex V_large_defocus_Cython_fake_adjustableKmax(int m,int n,double r,double f,int kmax):
    cdef:
        int p=(n-abs(m))/2
        int q=(n+abs(m))/2
        double complex res=0+0j
    res=epsilon(m)*exp(1j*f/2)*OuterSum_large_defocus_Cython_fake(m,n,r,f,kmax)
    return res 

cdef double complex OuterSum_large_defocus_Cython_fake(int m,int n,double r,double f,int kmax):
    cdef:
        int k_end=kmax+1
        double complex sum_value=0+0j
    for k in range(k_end):

            sum_value+=OuterSum_large_defocus_help_function_Cython_semi_fake(m,n,k,r,f)
    return sum_value

cdef double complex OuterSum_large_defocus_help_function_Cython_semi_fake(int m,int n,int k,double r,double f):
    cdef:
        double InnerSum_result=InnerSum_large_defocus_Cython(m,n,k,r)
        double complex dc 
    dc=(2*k+1)*((1j)**k)*gsl_Neven.gsl_sf_bessel_jl(k,f/2)*InnerSum_result
    return dc

'''
cdef double InnerSum_large_defocus_Cython(int m,int n,int k,double r):
    cdef:
        int p=(n-abs(m))/2
        int q=int((n+abs(m))/2)
        int lower_values_for_InnerSum=max([0,k-q,p-k])
        int l
        int l_end = k+p
        double[:] sum_coef = np.zeros(l_end+1)
        
    for l in range(lower_values_for_InnerSum,l_end+1):
        sum_coef[l]=InnerSum_large_defocus_help_function_Cython(m,n,k,l,p,r)
    return sum(sum_coef)
'''

cdef double InnerSum_large_defocus_Cython(int m,int n,int k,double r):
    cdef:
        int p=int((n-abs(m))/2)
        int q=int((n+abs(m))/2)
        int lower_values_for_InnerSum=max([0,k-q,p-k])
        int l
        int l_end = k+p
        double sum_value=0
        
    for l in range(lower_values_for_InnerSum,l_end+1):
        sum_value+=InnerSum_large_defocus_help_function_Cython(m,n,k,l,p,r)
    return sum_value

@cython.cdivision(True)
cdef double InnerSum_large_defocus_help_function_Cython(int m,int n,int k,int l,int p,double r):
    cdef:
        double pi=3.14159265358979323846
    res=((-1)**l)*omega_Cython(m,n,k,l)*gsl_Neven.gsl_sf_bessel_Jn(m+2*l+1,2*pi*r)/((2*pi*r))
    return res

'''
cdef double omega_Cython(int m,int n,int k,int l):
    cdef:
        int p=(n-abs(m))/2
        int s
        int s_end=p
        double res
        double[:] sum_coef = np.zeros(s_end+1)
    for s in range(0,s_end+1):
        sum_coef[s]=omega_sum_Cython(m,n,k,l,s)
    return sum(sum_coef)
'''

cdef double omega_Cython(int m,int n,int k,int l):
    cdef:
        int p=int((n-abs(m))/2)
        int s
        int s_end=p
        double res
        double sum_value=0
    for s in range(0,s_end+1):
        sum_value+=omega_sum_Cython(m,n,k,l,s)
    return sum_value

#cpdef double omega_sum_Cython(int m,int n,int k,int l,int s):
#    cdef: 
#        int t
#        int t_end=min(k,s)
#        double[:] sum_coef = np.zeros(t_end+1)
#    for t in range(0,t_end+1):
#        sum_coef[t]=omega_inner_sum_Cython(m,n,k,l,s,t)
#    return sum(sum_coef)

cdef double omega_sum_Cython(int m,int n,int k,int l,int s):
    cdef: 
        int t
        int t_end=min(k,s)
        double sum_value=0
    for t in range(0,t_end+1):
        sum_value+=omega_inner_sum_Cython(m,n,k,l,s,t)
    return sum_value

cdef double omega_inner_sum_Cython(int m,int n,int k,int l,int s,int t):
    cdef:
        int p
        double res
    p=int((n-abs(m))/2)          
    res=f_fun_Cython(m,p,s)*g_fun_Cython(m,k+s-2*t,l)*b_fun_Cython(k,s,t)
    return res

@cython.cdivision(True)
cdef double f_fun_Cython(int m,int p,int s):
    cdef:
        double nominator=(2*s+1)
        double denominator=(p+s+1)
    if m==0:
        if p==s:
            return 1
        else:
            return 0
    else:
        return (-1)**(p-s)*(nominator/denominator)*binomialCoeff_Cython(m+p-s-1,m-1)*binomialCoeff_Cython(m+p+s,s)/binomialCoeff_Cython(p+s,s)   
    
@cython.cdivision(True) 
cdef double g_fun_Cython(int m,int u,int l):
    cdef:
        double nominator=(m+2*l+1)
        double denominator=(m+u+l+1)
    if m==0:
         if u==l:
            return 1
         else:
            return 0
    else:
        if u>=l:
            return (nominator/denominator)*binomialCoeff_Cython(m,u-l)*binomialCoeff_Cython(u+l,l)/binomialCoeff_Cython(m+l+u,m+l)
        else:
            return 0
    
@cython.cdivision(True)
cdef double b_fun_Cython(int s1, int s2,int t):
    cdef:
        double nominator=2*s1+2*s2-4*t+1
        double denominator=2*s1+2*s2-2*t+1
    res=(nominator/denominator)*binomialCoeff_Cython(2*(s1-t),s1-t)*binomialCoeff_Cython(2*t,t)*binomialCoeff_Cython(2*(s2-t),s2-t)/binomialCoeff_Cython(2*(s1+s2-t),s1+s2-t)
    return res


In [17]:
n=2
m=2
%timeit sub_sum_Cython(1.1,n,m,10,0,20)

805 ns ± 13.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
